In [31]:
#from google.colab import drive
#drive.mount('/content/drive')

import argparse
import csv
import os
import sys
import re
import argparse
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from scipy.stats import loguniform
from scipy.io import loadmat
from sklearn import metrics
from torch.utils import data
from IPython.display import clear_output

#sys.path.append('/content/drive/MyDrive/Tesi_Magistrale')
sys.path.append('/content/drive/MyDrive/Tesi_Magistrale')
sys.path.append('D:/Tesi_Magistrale')
from Data_Processing_Utils import Norm_each_sub_by_own_param,NormalizeData, train_val_test_split_df, PlotLoss, plot_confusion_matrix, \
    Get_Sub_Norm_params, windowing_Dataframe

from DATALOADERS import dataframe_dataset_triplet, Pandas_Dataset

from MODELS import MKCNN, MKCNN_grid, random_choice, train_model_triplet, pre_train_model_triplet, train_model_standard, MultiKernelConv2D_grid

pd.options.mode.chained_assignment = None

In [ ]:
#file_path = '/content/drive/MyDrive/Tesi_Magistrale/'
#data_path = '/content/drive/MyDrive/Tesi_Magistrale/DATA/'

file_path = 'D:/Tesi_Magistrale'
data_path = 'D:/Tesi_Magistrale/DATA/'

sub_dir_list = [x for x in os.listdir(data_path) if 'offset_relabel' in x]
sub_days_dir_list = []
sub_day_data_dir_list = []
current_day = []

k = 0
print("Datasets found:")
for i in range(len(sub_dir_list)):
    print(f"{i+1}. {sub_dir_list[i]}")
    sub_days_dir_list.append([x for x in os.listdir(data_path+sub_dir_list[i]+'/') if 'day' in x and not 'merged' in x])
    for j in range(len(sub_days_dir_list[i])):
      print(f"\t{i+1}.{j+1}. {sub_days_dir_list[i][j]}")
      current_day_list = [x for x in os.listdir(data_path+sub_dir_list[i]+'/'+sub_days_dir_list[i][j]+'/csv/') if '6rep' in x]
      current_day.append(current_day_list)
      for f in current_day_list:
        print(f"\t\t{i+1}.{j+1}.{k+1}. {f}")
        k+=1
      k=0
    sub_day_data_dir_list.append(current_day)
    current_day = []
    j=0

del i
del j
del k
del current_day
del current_day_list


In [20]:
len(sub_day_data_dir_list[0][0])

5

In [42]:
dfs = []
prev_df_lenghts = 0
i = 0
day = 1
for subj in range(1,len(sub_day_data_dir_list)+1):
    for angle in range(1,len(sub_day_data_dir_list[subj-1][day-1])+1):
        current_df_path = data_path + sub_dir_list[subj-1]+'/'+sub_days_dir_list[subj-1][day-1]+'/csv/' + [ x for x in sub_day_data_dir_list[subj-1][day-1] if f'angle_{angle}' in x][0]
        print(f"Loading Sub {subj}, day {day}, angle {angle}")
        df = pd.read_csv(current_df_path)
        # prepare dataset for windowing
        # sub 2 acquisition are on left hand. Swap channels to compare it with right hands
        if subj == 2:
            df.rename(columns = {'Channel_1' : 'Ch8',
                'Channel_2' : 'Ch7',
                'Channel_3' : 'Ch6',
                'Channel_4' : 'Ch5',
                'Channel_5' : 'Ch4',
                'Channel_6' : 'Ch3',
                'Channel_7' : 'Ch2',
                'Channel_8' : 'Ch1',
                'Relabelled_repetition' : 'Rep_Num',
                'Relabelled_stimulus' : 'Exe_Num',
                }, inplace = True)
        else:
            df.rename(columns = {'Channel_1' : 'Ch1',
                'Channel_2' : 'Ch2',
                'Channel_3' : 'Ch3',
                'Channel_4' : 'Ch4',
                'Channel_5' : 'Ch5',
                'Channel_6' : 'Ch6',
                'Channel_7' : 'Ch7',
                'Channel_8' : 'Ch8',
                'Relabelled_repetition' : 'Rep_Num',
                'Relabelled_stimulus' : 'Exe_Num',
                }, inplace = True)
        df = df.loc[df['Exe_Num'] != 0]
        df['Exe_Num'] = df['Exe_Num']-1
        df['Sub'] = [subj]*len(df)
        df.drop(['Repetition', 'Stimulus'], axis = 1, inplace = True)
        df = windowing_Dataframe(df, 200, overlap_perc=0.5, NinaPro_Database_Num=None, drop_last=False, keep_angle = True, savefile = False)
        df.reset_index(inplace = True)
        df['sample_index'] = df['sample_index'] + prev_df_lenghts + i
        df['angle_index'] = [angle]*len(df)
        df['day'] = [day]*len(df)
        cols = [f'ch{c}' for c in range(1,9)]
        df[cols] = df[cols].astype(np.uint16)
        df['angle'] = df['angle'].astype(np.int16)
        
        dfs.append(df)
        prev_df_lenghts = df['sample_index'].iloc[-1]
        i=1
    
#concatenate all single dfs
del prev_df_lenghts
del i
df = pd.concat(dfs)
del dfs
del subj
del day
del angle
clear_output()
if not os.path.exists(data_path + f'Multi_Sub_dataset/day{day}/Dataframe/'):
    os.makedirs(data_path + f'Multi_Sub_dataset/day{day}/Dataframe/')
df.to_csv(data_path + f'Multi_Sub_dataset/day{day}/Dataframe/multi_sub_multi_day_wnd_200.csv')


AttributeError: module 'ntpath' has no attribute 'exist'

In [38]:
np.unique(df['day'])

array([1], dtype=int64)

In [ ]:
"""
dfs = []
prev_df_lenghts = 0
i = 0
for subj in range(1,len(sub_day_data_dir_list)+1):
    for day in range(1,len(sub_day_data_dir_list[subj-1])+1):
        for angle in range(1,len(sub_day_data_dir_list[subj-1][day-1])+1):
            current_df_path = data_path + sub_dir_list[subj-1]+'/'+sub_days_dir_list[subj-1][day-1]+'/csv/' + [ x for x in sub_day_data_dir_list[subj-1][day-1] if f'angle_{angle}' in x][0]
            print(f"Loading Sub {subj}, day {day}, angle {angle}")
            df = pd.read_csv(current_df_path)
            
            # prepare dataset for windowing
            # sub 2 acquisition are on left hand. Swap channels to compare it with right hands
            if subj == 2:
                df.rename(columns = {'Channel_1' : 'Ch8',
                    'Channel_2' : 'Ch7',
                    'Channel_3' : 'Ch6',
                    'Channel_4' : 'Ch5',
                    'Channel_5' : 'Ch4',
                    'Channel_6' : 'Ch3',
                    'Channel_7' : 'Ch2',
                    'Channel_8' : 'Ch1',
                    'Relabelled_repetition' : 'Rep_Num',
                    'Relabelled_stimulus' : 'Exe_Num',
                    }, inplace = True)
            else:
                df.rename(columns = {'Channel_1' : 'Ch1',
                    'Channel_2' : 'Ch2',
                    'Channel_3' : 'Ch3',
                    'Channel_4' : 'Ch4',
                    'Channel_5' : 'Ch5',
                    'Channel_6' : 'Ch6',
                    'Channel_7' : 'Ch7',
                    'Channel_8' : 'Ch8',
                    'Relabelled_repetition' : 'Rep_Num',
                    'Relabelled_stimulus' : 'Exe_Num',
                    }, inplace = True)
            df = df.loc[df['Exe_Num'] != 0]
            df['Exe_Num'] = df['Exe_Num']-1
            df['Sub'] = [subj]*len(df)
            df.drop(['Repetition', 'Stimulus'], axis = 1, inplace = True)
            df = windowing_Dataframe(df, 200, overlap_perc=0.5, NinaPro_Database_Num=None, drop_last=False, keep_angle = True, savefile = False)
            df.reset_index(inplace = True)
            df['sample_index'] = df['sample_index'] + prev_df_lenghts + i
            df['angle_index'] = [angle]*len(df)
            df['day'] = [day]*len(df)
            cols = [f'ch{c}' for c in range(1,9)]
            df[cols] = df[cols].astype(np.uint16)
            df['angle'] = df['angle'].astype(np.int16)
            
            dfs.append(df)
            prev_df_lenghts = df['sample_index'].iloc[-1]
            i=1
        
#concatenate all single dfs
del prev_df_lenghts
del i
df = pd.concat(dfs)
del dfs
del subj
del day
del angle
"""

In [39]:
# Normalize

NORM_MODE = 'sub'
NORM_TYPE = 'zero_to_one'
RECT = True

n_sub = np.unique(df['sub'])
n_day = np.unique(df['day'])
dfs = []
df_idx = df.set_index(['sub','day'])

for sub in n_sub:
  for day in n_day:
    try:
      current_df = df_idx.loc[sub, day]
      print(f"Normalizing Sub {sub}, day {day}")
      chs = current_df[cols]
      M = chs.values.max()
      m = chs.values.min()
      chs = (chs-m)/(M-m)
      current_df[cols] = chs[cols]
      dfs.append(current_df.reset_index())
    except KeyError:
      pass
df = pd.concat(dfs, axis = 0)
del dfs
del M
del m
del chs
del current_df
del sub
del day

Normalizing Sub 1, day 1
Normalizing Sub 2, day 1
Normalizing Sub 3, day 1


In [40]:
df.to_csv(data_path + 'Multi_Sub_dataset/multi_sub_multi_day_wnd_200_zero_to_one.csv')

In [3]:
single_model = True
data_path = '/content/drive/MyDrive/Tesi_Magistrale/DATA/'
NORM_MODE = 'sub'
NORM_TYPE = 'zero_to_one'
RECT = True
train_val_test_rep = [[2, 4, 6], [1, 5], [3]]
test_rep = train_val_test_rep[2][0]
WND_LEN = 200
print(f"Single Model: {single_model}\nTest Rep: {test_rep}")
exe_labels = ['medium_wrap', 'lateral', 'power_sphere', 'power_disk', 'prismatic_pinch', 'index_extension','wave_out', 'wave_in', 'fist', 'open_hand']
df = pd.read_csv(data_path + f"/Multi_Sub_dataset/multi_sub_multi_day_wnd_{WND_LEN}_zero_to_one.csv")
print("Dataset Loaded")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Single Model: True
Test Rep: 3
Dataset Loaded
Using device: cpu


In [4]:
n_channels = [cx for cx in df.columns if 'ch' in cx]
df.loc[:, n_channels] = df.loc[:, n_channels].astype(np.float32)
split_mode = 'rep'

if single_model:
    df_train, df_val, df_test = train_val_test_split_df(df, mode=split_mode, manual_sel=train_val_test_rep)
    valid_group = np.unique(df_val[split_mode])
    test_group = np.unique(df_test[split_mode])[0]
    train_set = Pandas_Dataset(df_train.groupby('sample_index'))
    valid_set = Pandas_Dataset(df_val.groupby('sample_index'))
    test_set = Pandas_Dataset(df_test.groupby('sample_index'))
    df_test.set_index('sample_index', inplace = True)

    # %% Dataloaders
    batch_size = 32
    num_workers = 0
    params = {'batch_size': batch_size,
            'shuffle': True,
            # 'sampler': sampler,
            'num_workers': num_workers,
            'drop_last': False}

    train_set_generator = data.DataLoader(train_set, **params)
    valid_set_generator = data.DataLoader(valid_set, **params)
    test_set_generator = data.DataLoader(test_set, **params)

    #%% Parametric model
    kernels_gap = [g for g in range(0, 3 * round(WND_LEN / 20), round(WND_LEN / 20))]
    kernel_sizes = np.full((3, 5, 2), [1, 3])
    for j in range(3):
        for i in range(5):
            kernel_sizes[j][i][0] = (round(WND_LEN / 20) * (i + 1) + kernels_gap[j])

    n_classes = 10

    net = {'N_multik': 32, 'N_Conv_conc': 64, 'N_SepConv': 128,
    'Kernel_multi_dim': kernel_sizes[0],
    'Kernel_Conv_conc': 1,
    'act_func': nn.ReLU(),
    'Pool_Type': nn.MaxPool2d,
    'wnd_len':WND_LEN
    }

    model = MKCNN_grid(net, num_classes=n_classes)
    model = model.to(device)

    #%% Loss Optim and scheduler
    # Define Loss functions
    cross_entropy_loss = nn.CrossEntropyLoss(reduction='mean').to(device)
    # triplet = nn.TripletMarginLoss(reduction='mean', margin=1, p=2)

    # Define Optimizer
    learning_rate = 0.0001
    # changed beta values from (0.5,0.999) to (0.9,0.999)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), weight_decay=1e-4)

    # # Define Scheduler
    precision = 1e-6
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, factor=.2,
                                                        patience=5, verbose=True, eps=precision)

    print(f'Model created with Num params: {sum([p.numel() for p in model.parameters() if p.requires_grad])}')

    best_weights, tr_losses, val_losses = train_model_standard(model=model, loss_fun=cross_entropy_loss,
                                                            optimizer=optimizer, scheduler=scheduler,
                                                            dataloaders={"train": train_set_generator,
                                                                            "val": valid_set_generator},
                                                            num_epochs=50, precision=precision,
                                                            patience=10, patience_increase=5)

    database = data_path + f"/Multi_Sub_dataset/test_rep{test_rep}/"
    filename = 'multi_sub_multi_day_wnd_200_zero_to_one'
    # Save state dict of the model
    if not os.path.exists(database + f'Cross_{split_mode}/Best_States/'):
        os.makedirs(database + f'Cross_{split_mode}/Best_States/')
    torch.save(best_weights['state_dict'],
            database + f'Cross_{split_mode}/Best_States/state_dict_{filename}.pth')

    # %% PlotLoss
    if not os.path.exists(database + f'Cross_{split_mode}/Plot/'):
        os.makedirs(database + f'Cross_{split_mode}/Plot/')

    PlotLoss(tr_losses, val_loss=val_losses,
            title=f'Cross-{split_mode}',
            path_to_save=database + f'Cross_{split_mode}/Plot/',
            filename=f'Cross_{split_mode}_{filename}.png')

    # %% Conf Matrix
    if not os.path.exists(database + f'Cross_{split_mode}/Conf_Matrix/'):
        os.makedirs(database + f'Cross_{split_mode}/Conf_Matrix/')

    # Evaluation
    softmax_block = nn.Softmax(dim=1)
    y_true = []
    y_pred = []

    test_dataloader = test_set_generator
    net = model
    # Load Weights
    net.load_state_dict(torch.load(database + f'Cross_{split_mode}/Best_States/state_dict_{filename}.pth'))

    net.eval()
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            # inputs = torch.swapaxes(inputs, 2, 1)  # -> convert from [10,20] to [20,10]
            inputs = inputs[:, None, :, :]
            inputs = inputs.to(device)
            labels_np = labels.cpu().data.numpy()
            # forward
            outputs, _ = net(inputs)
            outputs_np = softmax_block(outputs)
            outputs_np = outputs_np.cpu().data.numpy()
            outputs_np = np.argmax(outputs_np, axis=1)

            y_pred = np.append(y_pred, outputs_np)
            y_true = np.append(y_true, labels_np)

        cm = metrics.confusion_matrix(y_true=y_true, y_pred=y_pred)
        # Fancy confusion matrix
        plot_confusion_matrix(cm, target_names=exe_labels, title=f'Confusion Matrix for {split_mode} {test_group}',
                            path_to_save=database + f'Cross_{split_mode}/Conf_Matrix/Cross_{split_mode}_N_{filename}.png')

    # %% Write in csv cross_sub results
    # Building columns
    header_net = [f'Tested_{split_mode}', 'Best Val Loss', 'Accuracy', 'Kappa', 'F1_score', 'Best Epoch', 'Norm_Type',
                'Norm_Mode', 'Rect', f'Valid_{split_mode}']

    # Open the CSV file and write the headers and row of values
    with open(database + f'Cross_{split_mode}/Evals_single_model.csv', 'a', newline='') as myFile:
        writer = csv.writer(myFile)
        if myFile.tell() == 0:
            writer.writerow(header_net)
        # Create the row of values
        row = [test_group, min(val_losses),
            metrics.accuracy_score(y_true=y_true, y_pred=y_pred),
            metrics.cohen_kappa_score(y1=y_true, y2=y_pred, weights='quadratic'),
            metrics.f1_score(y_true=y_true, y_pred=y_pred, average='macro'),
            best_weights['epoch'], NORM_TYPE, NORM_MODE, str(RECT), valid_group]
        writer.writerow(row)
        print(f'Results Saved in -> {database}/Cross_{split_mode}/Evals.csv')

else:
    for ANGLE in range(1,6):
      current_df = df[df['angle_index'] == ANGLE]
    df_train, df_val, df_test = train_val_test_split_df(df, mode=split_mode, manual_sel=train_val_test_rep)
    valid_group = np.unique(df_val[split_mode])
    test_group = np.unique(df_test[split_mode])[0]
    train_set = Pandas_Dataset(df_train.groupby('sample_index'))
    valid_set = Pandas_Dataset(df_val.groupby('sample_index'))
    test_set = Pandas_Dataset(df_test.groupby('sample_index'))
    df_test.set_index('sample_index', inplace = True)

    # %% Dataloaders
    batch_size = 32
    num_workers = 0
    params = {'batch_size': batch_size,
            'shuffle': True,
            # 'sampler': sampler,
            'num_workers': num_workers,
            'drop_last': False}

    train_set_generator = data.DataLoader(train_set, **params)
    valid_set_generator = data.DataLoader(valid_set, **params)
    test_set_generator = data.DataLoader(test_set, **params)

    #%% Parametric model
    kernels_gap = [g for g in range(0, 3 * round(WND_LEN / 20), round(WND_LEN / 20))]
    kernel_sizes = np.full((3, 5, 2), [1, 3])
    for j in range(3):
        for i in range(5):
            kernel_sizes[j][i][0] = (round(WND_LEN / 20) * (i + 1) + kernels_gap[j])

    n_classes = 10

    net = {'N_multik': 32, 'N_Conv_conc': 64, 'N_SepConv': 128,
    'Kernel_multi_dim': kernel_sizes[0],
    'Kernel_Conv_conc': 1,
    'act_func': nn.ReLU(),
    'Pool_Type': nn.MaxPool2d,
    'wnd_len':WND_LEN
    }

    model = MKCNN_grid(net, num_classes=n_classes)
    model = model.to(device)

    #%% Loss Optim and scheduler
    # Define Loss functions
    cross_entropy_loss = nn.CrossEntropyLoss(reduction='mean').to(device)
    # triplet = nn.TripletMarginLoss(reduction='mean', margin=1, p=2)

    # Define Optimizer
    learning_rate = 0.0001
    # changed beta values from (0.5,0.999) to (0.9,0.999)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), weight_decay=1e-4)

    # # Define Scheduler
    precision = 1e-6
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, factor=.2,
                                                        patience=5, verbose=True, eps=precision)

    print(f'Model created with Num params: {sum([p.numel() for p in model.parameters() if p.requires_grad])}')

    best_weights, tr_losses, val_losses = train_model_standard(model=model, loss_fun=cross_entropy_loss,
                                                            optimizer=optimizer, scheduler=scheduler,
                                                            dataloaders={"train": train_set_generator,
                                                                            "val": valid_set_generator},
                                                            num_epochs=50, precision=precision,
                                                            patience=10, patience_increase=5)

    database = data_path + f"/Multi_Sub_dataset/test_rep{test_rep}/"
    filename = f'multi_sub_multi_day_wnd_200_zero_to_one_angle{ANGLE}'
    # Save state dict of the model
    if not os.path.exists(database + f'Cross_{split_mode}/Best_States/'):
        os.makedirs(database + f'Cross_{split_mode}/Best_States/')
    torch.save(best_weights['state_dict'],
            database + f'Cross_{split_mode}/Best_States/state_dict_{filename}.pth')

    # %% PlotLoss
    if not os.path.exists(database + f'Cross_{split_mode}/Plot/'):
        os.makedirs(database + f'Cross_{split_mode}/Plot/')

    PlotLoss(tr_losses, val_loss=val_losses,
            title=f'Cross-{split_mode}',
            path_to_save=database + f'Cross_{split_mode}/Plot/',
            filename=f'Cross_{split_mode}_{filename}.png')

    # %% Conf Matrix
    if not os.path.exists(database + f'Cross_{split_mode}/Conf_Matrix/'):
        os.makedirs(database + f'Cross_{split_mode}/Conf_Matrix/')

    # Evaluation
    softmax_block = nn.Softmax(dim=1)
    y_true = []
    y_pred = []

    test_dataloader = test_set_generator
    net = model
    # Load Weights
    net.load_state_dict(torch.load(database + f'Cross_{split_mode}/Best_States/state_dict_{filename}.pth'))

    net.eval()
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            # inputs = torch.swapaxes(inputs, 2, 1)  # -> convert from [10,20] to [20,10]
            inputs = inputs[:, None, :, :]
            inputs = inputs.to(device)
            labels_np = labels.cpu().data.numpy()
            # forward
            outputs, _ = net(inputs)
            outputs_np = softmax_block(outputs)
            outputs_np = outputs_np.cpu().data.numpy()
            outputs_np = np.argmax(outputs_np, axis=1)

            y_pred = np.append(y_pred, outputs_np)
            y_true = np.append(y_true, labels_np)

        cm = metrics.confusion_matrix(y_true=y_true, y_pred=y_pred)
        # Fancy confusion matrix
        plot_confusion_matrix(cm, target_names=exe_labels, title=f'Confusion Matrix for {split_mode} {test_group}',
                            path_to_save=database + f'Cross_{split_mode}/Conf_Matrix/Cross_{split_mode}_N_{filename}.png')

    # %% Write in csv cross_sub results
    # Building columns
    header_net = [f'Tested_{split_mode}', 'Best Val Loss', 'Accuracy', 'Kappa', 'F1_score', 'Best Epoch', 'Norm_Type',
                'Norm_Mode', 'Rect', f'Valid_{split_mode}']

    # Open the CSV file and write the headers and row of values
    with open(database + f'Cross_{split_mode}/Evals_multi_model_angle_{ANGLE}.csv', 'a', newline='') as myFile:
        writer = csv.writer(myFile)
        if myFile.tell() == 0:
            writer.writerow(header_net)
        # Create the row of values
        row = [test_group, min(val_losses),
            metrics.accuracy_score(y_true=y_true, y_pred=y_pred),
            metrics.cohen_kappa_score(y1=y_true, y2=y_pred, weights='quadratic'),
            metrics.f1_score(y_true=y_true, y_pred=y_pred, average='macro'),
            best_weights['epoch'], NORM_TYPE, NORM_MODE, str(RECT), valid_group]
        writer.writerow(row)
        print(f'Results Saved in -> {database}/Cross_{split_mode}/Evals.csv')


<ipython-input-4-595caa60dd6d>:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, n_channels] = df.loc[:, n_channels].astype(np.float32)


Model created with Num params: 108746
###################################################################### 




 Epoch 0/49 ---------------------------------------------------------------------- 





train -> loss: 2.24988, accuracy: 0.12500, kappa score: -0.0412, Epoch: -> 0 / 50  Batch Number: -> 1 / 1157
train -> loss: 2.29627, accuracy: 0.07812, kappa score: -0.1637, Epoch: -> 0 / 50  Batch Number: -> 2 / 1157
train -> loss: 2.31867, accuracy: 0.06250, kappa score: -0.1679, Epoch: -> 0 / 50  Batch Number: -> 3 / 1157
train -> loss: 2.33008, accuracy: 0.07031, kappa score: -0.1063, Epoch: -> 0 / 50  Batch Number: -> 4 / 1157
train -> loss: 2.31948, accuracy: 0.08750, kappa score: -0.0588, Epoch: -> 0 / 50  Batch Number: -> 5 / 1157
train -> loss: 2.32041, accuracy: 0.07812, kappa score: -0.0791, Epoch: -> 0 / 50  Batch Number: -> 6 / 1157
train -> loss: 2.33340, accuracy: 0.08036, kappa score: -0.0353, Epoch: -> 0 / 50  Batch Number: -> 7 / 1157
train -> loss: 2.33271, accuracy: 

KeyboardInterrupt: ignored